In [ ]:
%pip install ens-normalize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Normalize an ENS name:

In [ ]:
from ens_normalize import ens_normalize
# str -> str
# raises DisallowedNameError for disallowed names
# output ready for namehash
ens_normalize('Nick.ETH')
# 'nick.eth'
# note: does not enforce .eth TLD 3-character minimum

'nick.eth'

Inspect issues with names that cannot be normalized:

In [ ]:
from ens_normalize import DisallowedNameError, CurableError
# added a hidden "zero width joiner" character
try:
    ens_normalize('Ni‍ck.ETH')
# Catch the first normalization error (the name we are attempting to normalize could have more than one error).
except DisallowedNameError as e:
    # error code
    print(e.code)
    # INVISIBLE

    # a message about why the input is disallowed
    print(e.general_info)
    # Contains a disallowed invisible character

    if isinstance(e, CurableError):
        # information about the disallowed substring
        print(e.disallowed_sequence_info)
        # 'This invisible character is disallowed'

        # starting index of the disallowed substring in the input string
        # (counting in Unicode code points)
        print(e.index)
        # 2

        # the disallowed substring
        # (use repr() to "see" the invisible character)
        print(repr(e.disallowed))
        # '\u200d'

        # a suggestion for fixing the first error (there might be more errors)
        print(repr(e.suggested))
        # ''
        # replacing the disallowed substring with this empty string represents that the disallowed substring should be removed

        # You may be able to fix this error by replacing e.disallowed
        # with e.suggested in the input string.
        # Fields index, disallowed_sequence_info, disallowed, and suggested are not None only for fixable errors.
        # Other errors might be found even after applying this suggestion.

INVISIBLE
Contains a disallowed invisible character
This invisible character is disallowed
2
'\u200d'
''


You can attempt conversion of disallowed names into normalized names:

In [ ]:
from ens_normalize import ens_cure
# input name with disallowed zero width joiner and '?'
# str -> str
ens_cure('Ni‍ck?.ETH')
# 'nick.eth'
# ZWJ and '?' are removed, no error is raised
# note: this function is not a part of the ENS Normalization Standard

'nick.eth'

In [ ]:
# note: might still raise DisallowedNameError for certain names, which cannot be cured, e.g.
try:
  ens_cure('?')
except DisallowedNameError as e:
  print(f'{type(e).__name__}: {e}')
# DisallowedNameError: The name is empty

DisallowedNameError: The name is empty


In [ ]:
try:
  ens_cure('0χх0.eth')
except DisallowedNameError as e:
  print(f'{type(e).__name__}: {e}')
# DisallowedNameError: Contains visually confusing characters that are disallowed

DisallowedNameError: Contains visually confusing characters from Cyrillic and Latin scripts


Format names with fully-qualified emoji:

In [ ]:
from ens_normalize import ens_beautify
# works like ens_normalize()
# output ready for display
ens_beautify('1⃣2⃣.eth')
# '1️⃣2️⃣.eth'

# note: normalization is unchanged:
# ens_normalize(ens_beautify(x)) == ens_normalize(x)
# note: in addition to beautifying emojis, ens_beautify converts the character 'ξ' (Greek lowercase 'Xi') to 'Ξ' (Greek uppercase 'Xi', a.k.a. the Ethereum symbol) in labels that contain no other Greek characters

'1️⃣2️⃣.eth'

Generate detailed label analysis:

In [ ]:
from ens_normalize import ens_tokenize
# str -> List[Token]
# always returns a tokenization of the input
ens_tokenize('Nàme‍🧙‍♂.eth')
# [TokenMapped(cp=78, cps=[110], type='mapped'),
#  TokenNFC(input=[97, 768], cps=[224], type='nfc'),
#  TokenValid(cps=[109, 101], type='valid'),
#  TokenDisallowed(cp=8205, type='disallowed'),
#  TokenEmoji(emoji=[129497, 8205, 9794, 65039],
#             input=[129497, 8205, 9794],
#             cps=[129497, 8205, 9794],
#             type='emoji'),
#  TokenStop(cp=46, type='stop'),
#  TokenValid(cps=[101, 116, 104], type='valid')]

[TokenMapped(cp=78, cps=[110], type='mapped'),
 TokenNFC(input=[97, 768], cps=[224], type='nfc'),
 TokenValid(cps=[109, 101], type='valid'),
 TokenDisallowed(cp=8205, type='disallowed'),
 TokenEmoji(emoji=[129497, 8205, 9794, 65039], input=[129497, 8205, 9794], cps=[129497, 8205, 9794], type='emoji'),
 TokenStop(cp=46, type='stop'),
 TokenValid(cps=[101, 116, 104], type='valid')]

For a normalizable name, you can find out how the input is transformed during normalization:

In [ ]:
from ens_normalize import ens_transformations
# Returns a list of transformations (substring -> string)
# that have been applied to the input during normalization.
# NormalizationTransformation has the same fields as CurableError:
# - code
# - general_info
# - disallowed_sequence_info
# - index
# - disallowed
# - suggested
ens_transformations('Nàme🧙‍♂️.eth')
# [NormalizationTransformation(code="MAPPED", index=0, disallowed="N", suggested="n"),
#  NormalizationTransformation(code="FE0F", index=4, disallowed="🧙‍♂️", suggested="🧙‍♂")]

[NormalizationTransformation(code="MAPPED", index=0, disallowed="N", suggested="n"),
 NormalizationTransformation(code="FE0F", index=4, disallowed="🧙‍♂️", suggested="🧙‍♂")]

An example normalization workflow:

In [ ]:
name = 'Nàme🧙‍♂️.eth'
try:
    normalized = ens_normalize(name)
    print('Normalized:', normalized)
    # Normalized: nàme🧙‍♂.eth
    # Success!

     # was the input transformed by the normalization process?
    if name != normalized:
        # Let's check how the input was changed:
        for t in ens_transformations(name):
            print(repr(t)) # use repr() to print more information
        # NormalizationTransformation(code="MAPPED", index=0, disallowed="N", suggested="n")
        # NormalizationTransformation(code="FE0F", index=4, disallowed="🧙‍♂️", suggested="🧙‍♂")
        #                              invisible character inside emoji ^
except DisallowedNameError as e:
    # Even if the name is invalid according to the ENS Normalization Standard,
    # we can try to automatically remove disallowed characters.
    try:
        print(ens_cure(name))
    except DisallowedNameError as e:
        # The name cannot be automatically fixed.
        print('Fatal error:', e)

Normalized: nàme🧙‍♂.eth
NormalizationTransformation(code="MAPPED", index=0, disallowed="N", suggested="n")
NormalizationTransformation(code="FE0F", index=4, disallowed="🧙‍♂️", suggested="🧙‍♂")


You can run many of the above functions at once. It is faster than running all of them sequentially.

In [ ]:
from ens_normalize import ens_process
# use only the do_* flags you need
ens_process("Nàme🧙‍♂️1⃣.eth",
    do_normalize=True,
    do_beautify=True,
    do_tokenize=True,
    do_transformations=True,
    do_cure=True,
)
# ENSProcessResult(
#   normalized='nàme🧙\u200d♂1⃣.eth',
#   beautified='nàme🧙\u200d♂️1️⃣.eth',
#   tokens=[...],
#   cured='nàme🧙\u200d♂1⃣.eth',
#   cures=[], # This is the list of cures that were applied to the input (in this case, none).
#   error=None, # This is the exception raised by ens_normalize().
#               # It is a DisallowedNameError or CurableError if the error is curable.
#   transformations=[
#     NormalizationTransformation(code="MAPPED", index=0, disallowed="N", suggested="n"),
#     NormalizationTransformation(code="FE0F", index=4, disallowed="🧙‍♂️", suggested="🧙‍♂")
#   ])

ENSProcessResult(normalized='nàme🧙\u200d♂1⃣.eth', beautified='nàme🧙\u200d♂️1️⃣.eth', tokens=[TokenMapped(cp=78, cps=[110], type='mapped'), TokenValid(cps=[224, 109, 101], type='valid'), TokenEmoji(emoji=[129497, 8205, 9794, 65039], input=[129497, 8205, 9794, 65039], cps=[129497, 8205, 9794], type='emoji'), TokenEmoji(emoji=[49, 65039, 8419], input=[49, 8419], cps=[49, 8419], type='emoji'), TokenStop(cp=46, type='stop'), TokenValid(cps=[101, 116, 104], type='valid')], cured='nàme🧙\u200d♂1⃣.eth', cures=[], error=None, transformations=[NormalizationTransformation(code="MAPPED", index=0, disallowed="N", suggested="n"), NormalizationTransformation(code="FE0F", index=4, disallowed="🧙‍♂️", suggested="🧙‍♂")])